In [11]:
## Imports
import pandas as pd
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import torch

# Load the dataset once
file_path = 'eris_cleaned_no_dash.csv'  # Update with the correct file path
df = pd.read_csv(file_path)

# Define the mapping dictionaries for each label
mappings = {
    "context": {label: idx for idx, label in enumerate(df['context'].unique())},
    "motive": {label: idx for idx, label in enumerate(df['motive'].unique())},
    "consequence": {label: idx for idx, label in enumerate(df['longtermconsequence'].unique())},
    "level": {label: idx for idx, label in enumerate(df['level'].unique())},
}

# Model paths for each label
model_paths = {
    "context": './Knowledge_Extraction/saved_model_Roberta_large_context/',
    "motive": './Knowledge_Extraction/saved_model_Roberta_large_motive/',
    "consequence": './Knowledge_Extraction/saved_model_Roberta_large_consequence/',
    "level": './Knowledge_Extraction/saved_model_Roberta_large_level/',
}
# to sentence 2 add: "She took her life to avoid public shame."
# General function to classify new sentences for any label
def classify_new_sentences(sentences, model_path, label_name):
    # Load the tokenizer and model
    tokenizer = RobertaTokenizer.from_pretrained(model_path)
    model = RobertaForSequenceClassification.from_pretrained(model_path)

    # Tokenize the new sentences
    encodings = tokenizer(sentences, truncation=True, padding=True, return_tensors='pt')
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    encodings = {key: val.to(device) for key, val in encodings.items()}
    
    # Make predictions
    model.eval()
    with torch.no_grad():
        outputs = model(**encodings)
        logits = outputs.logits
    
    # Convert logits to probabilities and then to labels
    probs = torch.nn.functional.softmax(logits, dim=1)
    pred_labels = torch.argmax(probs, dim=1).cpu().numpy()
    
    # Map the predicted labels back to the original labels
    label_mapping = mappings[label_name]
    pred_classifications = [list(label_mapping.keys())[label] for label in pred_labels]
    
    return pred_classifications

# Example usage for all labels
def classify_sentences_across_models(sentences):
    predictions = {}
    
    for label_name, model_path in model_paths.items():
        pred_labels = classify_new_sentences(sentences, model_path, label_name)
        predictions[label_name] = pred_labels
    
    return predictions

# Test the function with multiple sentences
new_sentences = [
    "A lover stabs their partner in a fit of jealousy after discovering infidelity",
    "Fanatics burn down a rival sect’s temple with the priests inside.",
    "A gang of thieves kills a shop owner during a robbery for gold and goods.",
    "Ramses the second unleashed the full might of the chariots on Mageddo. This was recorded as the largest chariot battle in the bronze age",
    
]
predictions = classify_sentences_across_models(new_sentences)

# Print predictions in a simpler format
for i, sentence in enumerate(new_sentences):
    print(f"Sentence: {sentence}")
    for label_name, preds in predictions.items():
        print(f"{label_name.capitalize()}: {preds[i]}")
    print("-" * 50)


Sentence: A lover stabs their partner in a fit of jealousy after discovering infidelity
Context: conspiracy
Motive: emotional
Consequence: death
Level: interpersonal
--------------------------------------------------
Sentence: Fanatics burn down a rival sect’s temple with the priests inside.
Context: religious
Motive: political
Consequence: destruction/devastation
Level: intrasocial
--------------------------------------------------
Sentence: A gang of thieves kills a shop owner during a robbery for gold and goods.
Context: sack
Motive: economical
Consequence: plunder
Level: intersocial
--------------------------------------------------
Sentence: Ramses the second unleashed the full might of the chariots on Mageddo. This was recorded as the largest chariot battle in the bronze age
Context: war/military campaign
Motive: tactical/strategical
Consequence: victory
Level: intersocial
--------------------------------------------------
Sentence: Abel was brutally killed by his brother Cain so

## TRIAL 2